In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import pandas as pd
import numpy as np
import os
import sys
import pypinyin

In [4]:
# !pip install pypinyin

In [5]:
def check_chs(c):
    return '\u4e00' <= c <= '\u9fa5'

def to_pinyin(word):
    s = ''
    for i in pypinyin.pinyin(word, style=pypinyin.NORMAL):
        s += ''.join(i)
    return s

In [6]:
##
whole_pub_info = pd.read_pickle('./pkl/whole_pub_info.pkl')
whole_author_name_paper_ids = pd.read_pickle('./pkl/whole_author_name_paper_ids.pkl')
## 
train_pub_info = pd.read_pickle('./pkl/train_pub_info.pkl')
train_author_name_ids = pd.read_pickle('./pkl/train_author_name_ids.pkl')
train_author_paper_ids = pd.read_pickle('./pkl/train_author_paper_ids.pkl')
##
valid_pub_info = pd.read_pickle('./pkl/valid_pub_info.pkl')
cna_valid_unass = pd.read_pickle('./pkl/cna_valid_unass.pkl')

In [7]:
pub_info = pd.concat([whole_pub_info, train_pub_info, valid_pub_info])
print(pub_info.shape)
pub_info = pub_info.drop_duplicates(subset='paper_id', keep='first')
print(pub_info.shape)

(474711, 7)
(271111, 7)


In [8]:
whole_author_name_paper_ids.head()

,author_id,author_name,paper_ids
0,004mBKh6,changsheng_li,"[lRl7m2tU, 1vUnZLES, 5IbEOTRh, dVAKsHla, TY0MY..."
1,008b5hIc,yuanyuan_liu,[nPuf0kYs]
2,00BAgHFr,xi_li,"[TuEkjf5i, kn6wo3ot, 0f2awX7U]"
3,00M93JDI,weiping_cai,"[mHDfYkqE, RWISo90R, FSsnfI0X, 4U4GvMmc, Yh45T..."
4,00ShLPmg,hong_xiao,"[inFGp3mc, uyJus3zu, o6poFI1m, KvHObx7O, vg64a..."


In [9]:
paper_authors = {}
for author_name, paper_ids in whole_author_name_paper_ids[['author_name', 'paper_ids']].values:
    for pid in paper_ids:
        if not pid in paper_authors:
            paper_authors[pid] = [author_name]
        else:
            paper_authors[pid].append(author_name)

In [10]:
paper_authors_df = pd.DataFrame([(k, v) for k,v in paper_authors.items()], columns=['paper_id', 'author_ids'])

In [11]:
pub_info['author_names'] = pub_info['authors'].apply(lambda x: [ao['name'] for ao in x])

In [12]:
pub_info = pub_info.merge(paper_authors_df, 'left', 'paper_id')

In [13]:
pub_info.head()

,abstract,authors,keywords,paper_id,title,venue,year,author_names,author_ids
0,Extremely long states of bound solitons consis...,"[{'name': 'Ya Liu', 'org': 'Sch. of Electron. ...","[carbon nanotubes, fibre lasers, laser cavity ...",0009qJgC,Generating ultra-long bound soliton sequences ...,Lasers and Electro-Optics,2014,"[Ya Liu, Xin Zhao, Jiansheng Liu, Guoqing Hu]",[guoqing_hu]
1,NaN,"[{'name': 'Hao He'}, {'name': 'Jun Yang'}]","[different part, data graph, variety ofstructu...",000BMUBY,Multiresolution Indexing of XML for Frequent Q...,icde,2004,"[Hao He, Jun Yang]",[jun_yang]
2,The Yellow River (YR) supplies a large amount ...,"[{'name': 'Weifeng Yang', 'org': 'State Key La...","[13C, 15N, East China Sea, Trace element, Yell...",000Fmw78,Relocation of the Yellow River as revealed by ...,Marine Pollution Bulletin,2009,"[Weifeng Yang, Min Chen, Guangxue Li, Jianping...",[min_chen]
3,Our findings suggest that Th17 cells may be in...,"[{'name': 'Xiaowei Wang', 'org': 'Department o...","[Mycoplasma Pneumoniae, Th17 cells, children, ...",000HsdYy,Increased Frequency of Th17 Cells in Children ...,Journal of clinical laboratory analysis,2016,"[Xiaowei Wang, Xiaojun Chen, Heng Tang, Jifeng...",[feng_liu]
4,,"[{'name': 'jun chen', 'org': ''}, {'name': 'xi...",[],000NKgQT,Rapid Synthesis of Multiferroic BiFeO 3 Single...,Chemistry of Materials,2007,"[jun chen, xianran xing, andrew d watson, wei ...",[jun_chen]


In [16]:
from collections import defaultdict
def score(n1, n2):
    n1 = ''.join(filter(str.isalpha, n1.lower()))
    if check_chs(n1):
#         print(n1)
        n1 = to_pinyin(n1)
#         print(n1)
    n2 = ''.join(filter(str.isalpha, n2.lower()))
    counter = defaultdict(int)
    score = 0
    for c in n1:
        counter[c] += 1
    for c in n2:
        if (c in counter) and (counter[c] > 0):
            counter[c] -= 1
        else:
            score += 1
    score += np.sum(list(counter.values()))
    return score
    
score('hello world', 'world hello')

0

In [19]:
from tqdm import tqdm_notebook
author_name_map = {}
for author_names, author_ids in tqdm_notebook(pub_info[['author_names', 'author_ids']].values):
    if type(author_ids) == float:
        continue
    for aid in author_ids:
        dis = []
        for an in author_names:
            dis.append(score(an, aid))
        cor = author_names[np.argmin(dis)]
        author_name_map[cor] = aid

In [20]:
for k in author_name_map.keys():
    if check_chs(k):
        print(k)

王慧
刘建国 liu jianguo
刘晶 liu jing
胡斌


In [22]:
import pickle
with open('./pkl/author_name_map.pkl', 'wb') as file:
    pickle.dump(author_name_map, file)

In [25]:
author_name_map

{'Guoqing Hu': 'guoqing_hu',
 'Jun Yang': 'jun_yang',
 'Min Chen': 'min_chen',
 'Feng Liu': 'feng_liu',
 'jun chen': 'jun_chen',
 'Qing Zhang': 'qing_zhang',
 'Jie Yang': 'jie_yang',
 'F. Liu': 'f_liu',
 'Xiang Wang': 'xiang_wang',
 'ZHANG Tong': 'tong_zhang',
 'Lan Wang': 'lan_wang',
 'Jie Zhang': 'jie_zhang',
 'Ming Chen': 'ming_chen',
 'Long Wang': 'long_wang',
 'Hong Cheng': 'hong_cheng',
 'Wenping Hu': 'wenping_hu',
 'Gang Zhang': 'gang_zhang',
 'Li Jiang': 'li_jiang',
 'Zhang Hui': 'hui_zhang',
 'He Tian': 'he_tian',
 'haiying wang': 'haiying_wang',
 'xiaoming li': 'xiaoming_li',
 'Huan Liu': 'huan_liu',
 'CHEN Guo-hua': 'guohua_chen',
 'Zhang Bin': 'bin_zhang',
 'Jing Li': 'jing_li',
 'Wei Song': 'wei_song',
 'ZHAO Feng': 'feng_zhao',
 'Rui Zhang': 'rui_zhang',
 'Li Ma': 'li_ma',
 'Bo Zhou': 'bo_zhou',
 'Xiao Liu': 'xiao_liu',
 'Hui Xiong': 'hui_xiong',
 'Bo Shen': 'bo_shen',
 'Xin Li': 'xin_li',
 'Jie Xu': 'jie_xu',
 'C.-C. Wang': 'c_c_wang',
 'Sun Kai': 'kai_sun',
 'Cheng Zhu'

In [24]:
len(author_name_map)

2164